In [1]:
cd ../

In [2]:
!nvidia-smi

Wed May 31 18:37:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:02:00.0 Off |                    0 |
| N/A   28C    P0    25W / 250W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:82:00.0 Off |                    0 |
| N/A   

In [4]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
from torch.utils.data import DataLoader
from genetic_marker_dataset import GeneImageDataset
from torchvision import transforms
from experimental.network.embedding import SoftmaxEmbeddingNet
from tqdm.auto import tqdm

# RGB

In [5]:
image_transform = transforms.Compose([transforms.CenterCrop(1024),
                                      transforms.Resize(512),
                                      transforms.ToTensor()])
ds = GeneImageDataset(folder='/data/shared/genetic_marker_datasets/', 
                     sensor='rgb', 
                     transform=image_transform)
dl = DataLoader(ds, batch_size=30, shuffle=False, num_workers=10)


In [8]:
model = SoftmaxEmbeddingNet(375)
state_dict = torch.load('results/model/s9_pretrained_rgb.pth', map_location='cpu')

In [9]:
model.load_state_dict(state_dict['model_state_dict'], strict=False)

<All keys matched successfully>

In [10]:
ebd_model = model.backbone
ebd_model.cuda()
print('Model loaded on GPU')

Model loaded on GPU


In [ ]:
ebd_1024_list = []
ebd_128_list = []

def gap_layer_hook(module, input, output):
    ebd_1024_list.append(output.squeeze(-1).unsqueeze(-1).cpu().detach())
    
gap_layer_hook_handle = ebd_model.avgpool.register_forward_hook(gap_layer_hook)

ebd_model.eval()
with torch.no_grad():
    for img, label in tqdm(dl):
        ebd = ebd_model(img.cuda())
        ebd_128_list.append(ebd.cpu().detach())
    ebd_128 = torch.cat(ebd_128_list, dim=0)
    ebd_1024 = torch.cat(ebd_1024_list, dim=0)

gap_layer_hook_handle.remove()
save_path = 'results/snp_pred/s9_jpg_rgb_gene_ds_img_ebd.pth'
if not os.path.exists(os.path.dirname(save_path)):
    os.makedirs(os.path.dirname(save_path))
torch.save({'ebd_128': ebd_128, 'ebd_2048': ebd_1024},
            save_path)

# 3D

In [17]:
image_transform = transforms.Compose([transforms.CenterCrop(512),
                                      transforms.ToTensor()])
ds = GeneImageDataset(folder='/data/shared/genetic_marker_datasets/',
                      sensor='3d', 
                      transform=image_transform)
dl = DataLoader(ds, batch_size=30, shuffle=False, num_workers=10)


In [22]:
model = SoftmaxEmbeddingNet(375)
state_dict = torch.load('results/model/s9_pretrained_3d.pth', map_location='cpu')

In [23]:
model.load_state_dict(state_dict['model_state_dict'], strict=False)

<All keys matched successfully>

In [24]:
ebd_model = model.backbone
ebd_model.cuda()
print('Model loaded on GPU')

Model loaded on GPU


In [ ]:
ebd_1024_list = []
ebd_128_list = []

def gap_layer_hook(module, input, output):
    ebd_1024_list.append(output.squeeze().cpu().detach())
    
gap_layer_hook_handle = ebd_model.avgpool.register_forward_hook(gap_layer_hook)

ebd_model.eval()
with torch.no_grad():
    for img, label in tqdm(dl):
        ebd = ebd_model(img.cuda())
        ebd_128_list.append(ebd.cpu().detach())
    ebd_128 = torch.cat(ebd_128_list, dim=0)
    ebd_1024 = torch.cat(ebd_1024_list, dim=0)

gap_layer_hook_handle.remove()
save_path = 'results/snp_pred/s9_jpg_3d_gene_ds_img_ebd.pth'
if not os.path.exists(os.path.dirname(save_path)):
    os.makedirs(os.path.dirname(save_path))
torch.save({'ebd_128': ebd_128, 'ebd_1024': ebd_1024},
            save_path)